In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad, dblquad, trapz # Integration and double integration
from scipy.special import betainc, betaincinv # Useful for the normalisation coefficient in question 1
from scipy.stats import invgamma
from scipy.linalg import inv
from scipy.linalg import toeplitz
import time

In [6]:
l = 0
k = 100
T = 200
rho = 0.75
a, b, A, B = 1, 1, 1, 1

start = np.arange(0, 0.1 + 0.001, 0.001)[1:]
mid = np.arange(0.11, 0.9 + 0.01, 0.01)
end = np.arange(0.901, 1 + 0.001, 0.001)[:-1]
support = np.concatenate((start, mid, end))

In [7]:
def density_R2(r, z):
    s = np.sum(z)
    return ((1-r)/r)**s/2

def cdf_trapezoidal(x_values, y_values):
    return np.cumsum((x_values[1:] - x_values[:-1]) * (y_values[:-1] + y_values[1:]) / 2)

def posterior_R2(r, z):
    x_values = support
    y_values = [density_R2(x, z) for x in x_values]
    cdf_values = cdf_trapezoidal(x_values, y_values)
    normalization = cdf_values[-1]
    return np.interp(r, x_values, cdf_values) / normalization

def find_nearest_idx(value, array):
    idx = np.abs(array - value).argmin()
    return idx

def draw_R2_q(z, a, b, A, B):
    s = np.sum(z)
    a_1 = s + s/2 + a
    b_1 = k - s + b
    point = np.random.uniform(0, 1)
    CDF = np.array([posterior_R2(support[i], z) for i in range(len(support))])
    q_draw = betaincinv(a_1, b_1, point)
    nearest_index_point = find_nearest_idx(point, CDF)
    R2_draw = support[nearest_index_point]
    return (R2_draw, q_draw)

In [8]:
def sample_z ( Y, X, q, R_2, z ):
    
    zaza = np.copy(z)
    
    try :
        for i in range(k):

            v = np.mean(np.var(X, axis = 0))# Mean of the variances of each predictor (axis = 0 because a row of x is a regressor)
            gamma_2 = R_2/(q*k*v*(1-R_2))

            # In order to operate P(zi = 1 | Y,X,q, R_2, z_{-i}), we prepare two versions of every vectors, 
            #for each case where z_i = 1 and z_i = 0
            zeta, zeta_0 = np.copy(zaza), np.copy(zaza)
            if zeta[i] == 1:
                zeta_0[i] = 0
            else:
                zeta[i] = 1

            non_zeros_index = np.nonzero(zeta)[0]
            non_zeros_index_0 = np.nonzero(zeta_0)[0]

            # Here, for exemple, we compute X_tilde for the case where the i-th cooridinate of X is null or not
            X_tilde = X[ : , non_zeros_index]
            X_tilde_0 = X[ : , non_zeros_index_0]

            # Same for W_tilde
            W_tilde = (np.dot(X_tilde.T, X_tilde) + np.eye(len(non_zeros_index))/gamma_2)
            W_tilde_0 = (np.dot(X_tilde_0.T, X_tilde_0) + np.eye(len(non_zeros_index_0))/gamma_2)

            W_tilde_inv = inv(W_tilde)
            W_tilde_inv_0 = inv(W_tilde_0)

            beta_tilde_hat = np.dot(W_tilde_inv, np.dot(X_tilde.T, Y))
            beta_tilde_hat_0 = np.dot(W_tilde_inv_0, np.dot(X_tilde_0.T, Y))

            p = 1 / (1 + ( ((gamma_2**0.5)*(1-q)) / q ) * (np.linalg.det(W_tilde_inv_0)/np.linalg.det(W_tilde_inv))**(0.5) 
                           * ( (np.dot(Y.T,Y) - np.dot(beta_tilde_hat_0.T , np.dot(W_tilde_0, beta_tilde_hat_0))) 
                              / (np.dot(Y.T,Y) - np.dot(beta_tilde_hat.T , np.dot(W_tilde, beta_tilde_hat))))**(-T//2))

            u = np.random.uniform(0, 1)
            if u <= p:
                zaza[i] = 1
            else:
                zaza[i] = 0
    except ValueError :
        print(z)
        return np.copy(z)
    
    return zaza 

In [9]:
def draw_sigma(Y, X, beta, R_2, q, z):
    s = np.sum(z)
    non_zeros_index = np.nonzero(z)[0]
    X_tilde = X[ :, non_zeros_index]
    v = np.mean(np.var(X, axis = 0)) # Mean of the variances of each predictor (axis = 0 because a row of x is a regressor)
    gamma_2 = R_2/(q*k*v*(1-R_2))
    W_tilde = (np.dot(X_tilde.T, X_tilde) + np.eye(s)/gamma_2)
    W_tilde_inv = inv(W_tilde)
    beta_tilde_hat = np.dot(W_tilde_inv, np.dot(X_tilde.T, Y))
    return invgamma.rvs(T/2, scale=(np.dot(Y.T, Y) - np.dot(beta_tilde_hat.T, np.dot(W_tilde, beta_tilde_hat)))/2)

In [10]:
def draw_beta_tilde(Y, X, R_2, q, sigma_2, z):
    s = np.sum(z)
    non_zeros_index = np.nonzero(z)[0]
    X_tilde = X[ :, non_zeros_index]
    v = np.mean(np.var(X, axis = 0)) # Mean of the variances of each predictor (axis = 0 because a row of x is a regressor)
    gamma_2 = R_2/(q*k*v*(1-R_2))
    W_tilde = (np.dot(X_tilde.T, X_tilde) + np.eye(s)/gamma_2)
    W_tilde_inv = inv(W_tilde)
    mean = np.dot(W_tilde_inv, np.dot(X_tilde.T, Y))
    cov = sigma_2 * W_tilde_inv
    return np.random.multivariate_normal(mean, cov)

In [11]:
def generate_dataset(s, R_y, n_iterations = 500, burn_out = 100):
    
    # Generate a Toeplitz correlation matrix
    correlation_matrix = toeplitz(rho ** np.arange(k))

    # Generate random samples from a zero-mean Normal distribution
    mean = np.zeros(k)
    X = np.random.multivariate_normal(mean, correlation_matrix, size=T)
    v = np.mean(np.var(X, axis = 0))
    
    # We generate the first iteration of z
    z0 = np.zeros(100, dtype = int) 
    for i in np.random.choice(np.arange(k), s, replace = False) :
        z0[i] = 1 # We choose the s coordinates of Beta that are non zeros
    # Array to stock the data sets
    z_list = [z0]
    R_2_list = np.zeros(n_iterations)
    q_list = np.zeros(n_iterations)
    sigma_list = np.zeros(n_iterations)
    beta_tilde_list = []
    
    for i in range(n_iterations): # We iterate 5000 times, and then delete the 1000 first iterations (burn-in period). At the end, we obtain a dataset with 4000 draws
        
        # We now need to generate all the variables with each new "z"
        z = z_list[i]
        
        # Beta
        beta_non_zeros_index = np.nonzero(z)[0] 
        beta = np.zeros(k)
        beta[beta_non_zeros_index] = np.random.normal(0, 1, size=sum(z)) # We sample the s non zeros coordinates of Beta from a N(0, 1)
        
        # We then compute sigma epsilon in order to compute Y
        s_epsilon = 0
        for t in range(T): # We compute sigma_epsilon (It's different from sigma_2 !)
            s_epsilon += (np.dot(beta, X[t, :]))**2
        sigma_epsilon = s_epsilon/(T)*(1/R_y - 1)
        epsilon = np.random.normal(0, sigma_epsilon, size = T)
        # We can then actualise Y according to our new beta
        Y = np.dot(X, beta) + epsilon
        
        # Question 1
        R_2, q = draw_R2_q(z, a, b, A, B)
        R_2_list[i], q_list[i] = R_2, q
        # Question 3
        z_draw = sample_z ( Y, X, q, R_2, z )
        if np.array_equal(z_draw, z) == True :
            print(i)
        z_list.append( z_draw )
        # Question 4
        sigma_2 = draw_sigma (Y, X, beta, R_2, q, z)
        sigma_list[i] = sigma_2
        # Question 5
        beta_tilde_list.append( draw_beta_tilde(Y, X, R_2, q, sigma_2, z) )
    
    return q_list[burn_out:]

In [ ]:
from statistics import median
median_q_dataset_of_dataset = []
R_y_list = [0.05, 0.25, 0.5]
comptage = 4

for R_y in R_y_list :
    median_q_dataset = []
    for i in range(100):
        start = time.time()
        median_q_dataset.append(np.median(generate_dataset(10, R_y)))
        end = time.time()
        print("Time of execution =",((end-start)//60), "m", (((end-start)%60)//1), "; s=", 10, " Ry=", R_y, " iteration:", i)
    file_name = 'list_of_median_' + str(comptage) +'.csv'
    pd.DataFrame(np.array(median_q_dataset)).to_csv(file_name)
    median_q_dataset_of_dataset.append(median_q_dataset)
    comptage += 1